### Hi-C Data Preparation

1. Convert chromosome anchor.to.anchor files to full genome cooler file
2. Map cooler to uniform (10kb) bins
3. Convert to sparse matrix (.npz) format

In [ ]:
! python src/cshark/preprocessing/convert_to_cooler.py \
    --loop_dir /mnt/jinstore/Archive01/LAB/Hi-C/ssz20_12122022_publicArima_AlphaBetaAcinar/processed/beta/Enhance_250M \
    --anchors /mnt/jinstore/JinLab03/xxl1432/Reference/HiC/enzyme/hg19_GATC_GANTC/anchor_bed \
    --col_names a1 a2 ratio \
    --out beta_5kb_deeploop.cool

In [ ]:
! python src/cshark/preprocessing/cooler_uniform_bins.py beta_5kb_deeploop.cool beta_10kb_deeploop.cool 10000

In [ ]:
! python src/cshark/preprocessing/cool2npy.py beta_10kb_deeploop.mcool ../cshark_data/data/hg19/beta/hic_matrix --no-balance

### Data Organization

Training expects data organized according to:

`<data_root>/<assembly>/<celltype>`

Each of these celltype directories should contain a `genomic_features` folder containing the training and target bigwigs, as well as a `hic_matrix` folder containing the output of the Hi-C preprocessing pipeline.

An example of a celltype directory looks like this:

In [7]:
! tree ../cshark_data/data/hg19/beta

../cshark_data/data/hg19/beta
├── genomic_features
│   ├── atac.bw
│   └── ctcf.bw
└── hic_matrix
    ├── chr10.npz
    ├── chr11.npz
    ├── chr12.npz
    ├── chr13.npz
    ├── chr14.npz
    ├── chr15.npz
    ├── chr16.npz
    ├── chr17.npz
    ├── chr18.npz
    ├── chr19.npz
    ├── chr1.npz
    ├── chr20.npz
    ├── chr21.npz
    ├── chr22.npz
    ├── chr2.npz
    ├── chr3.npz
    ├── chr4.npz
    ├── chr5.npz
    ├── chr6.npz
    ├── chr7.npz
    ├── chr8.npz
    ├── chr9.npz
    ├── chrX.npz
    └── chrY.npz

3 directories, 26 files


### Training

Once the data is organized, you can run the `train.py` script with the arguments:

* `--data-root`

* `--assembly`

* `--celltypes`: can provide a single celltype folder name, or a whole list

* `--input-features`: names of bigwig files to use as input

You can optionally provide `--target-features` of the same or different bigwigs and the model will try to reconstruct/predict these.

In [ ]:
! python3 ../src/cshark/training/train.py \
    --data-root ../cshark_data/data \
    --assembly hg19 \
    --celltypes beta \
    --input-features ctcf atac \
    --target-features ctcf atac \
    --latent-dim 128 \
    --num-gpu 1 \
    --batch-size 2 \
    --num-workers 1 \
    --use-wandb